In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [3]:
columns = ['LoanStatus', 'EmploymentStatus', 'CreditScoreRangeLower',
           'CreditScoreRangeUpper', 'LoanOriginalAmount', 'IsBorrowerHomeowner',
            'CurrentDelinquencies', 'AmountDelinquent']

data_unclean = pd.read_csv('prosperLoanData.csv')
data_filtered = data_unclean[columns]
print(data_filtered.shape)

# for col in data_filtered:
#     print(col)
#     print(data_filtered[col].unique())

#Converting Loan Status into a binary classification
data_filtered['LoanStatus'] = data_filtered["LoanStatus"].apply(lambda x: 1 if x in["Completed", "Current", "FinalPaymentInProgress"] else 0)

#Converting Booleans to Int
data_filtered['IsBorrowerHomeowner'] = data_filtered['IsBorrowerHomeowner'].apply(lambda x: 1 if x == True else 0)

#Converting Employment Status into a binary classification
data_filtered['EmploymentStatus'] = data_filtered['EmploymentStatus'].apply(lambda x: 1 if x in ['Employed', 'Full-time', 'Self-employed', 'Part-time'] else 0)

data = data_filtered.dropna()

data['CreditScoreAvg'] = (data['CreditScoreRangeLower'] + data['CreditScoreRangeUpper'])/2
data.drop(columns=['CreditScoreRangeUpper', 'CreditScoreRangeLower'], inplace=True)

print(data)

(113937, 8)
        LoanStatus  EmploymentStatus  LoanOriginalAmount  IsBorrowerHomeowner  \
0                1                 1                9425                    1   
1                1                 1               10000                    0   
3                1                 1               10000                    1   
4                1                 1               15000                    1   
5                1                 1               15000                    1   
...            ...               ...                 ...                  ...   
113932           1                 1               10000                    1   
113933           1                 1                2000                    1   
113934           1                 1               10000                    1   
113935           1                 1               15000                    1   
113936           1                 1                2000                    0   

        Current

/var/folders/dg/qztys7fj1cg0sp52mvk7khkh0000gn/T/ipykernel_16392/3071364611.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['LoanStatus'] = data_filtered["LoanStatus"].apply(lambda x: 1 if x in["Completed", "Current", "FinalPaymentInProgress"] else 0)
/var/folders/dg/qztys7fj1cg0sp52mvk7khkh0000gn/T/ipykernel_16392/3071364611.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['IsBorrowerHomeowner'] = data_filtered['IsBorrowerHomeowner'].apply(lambda x: 1 if x == True else 

In [14]:
x_train, x_test, y_train, y_test = train_test_split(data.iloc[:, 1:data.shape[1]], data['LoanStatus'], train_size=0.75, shuffle=True, random_state=2)

print(x_test.shape)
print(x_train.shape)

y_train_dist = np.bincount(np.array(y_train))
y_test_dist = np.bincount(np.array(y_test))

print(y_train_dist[0]/y_train_dist[1]) 
print(y_test_dist[0]/y_test_dist[1]) # Checking if the distribution of delinquent loans in the test and train datasets is roughly equal

(26579, 6)
(79736, 6)
0.177436503248671
0.17595787983364303


In [15]:
rf = RandomForestClassifier(n_estimators=30, random_state=2)
# train the model
rf.fit(x_train, y_train)
# make predictions on train and test set
pred_train = rf.predict_proba(x_train)
pred_test = rf.predict_proba(x_test)
print('Training set results')
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1], multi_class='ovr')))
print('Test set results')
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1], multi_class='ovr')))

Training set results
Random Forests roc-auc: 0.8271910206061146
Test set results
Random Forests roc-auc: 0.6518149933304893
